# Context and Objectives

The advancement of industrial Internet of Things (IoT) and industrial Big Data Analytics has spurred huge promise of economic values through data-driven predictive modeling in a variety of industries. Fault detection and prognostics are common problems in industrial plant monitoring and are at the core of systems health management.  
One of the most important tasks in prognostics is Remaining Useful Life (RUL) prediction, in units of time (e.g., hours or cycles). The objective of RUL prediction is to estimate the remaining working time of a system before its health condition deteriorates below a threshold indicating a faulty condition. 

In the age of Internet of Things and Industrial 4.0, the prognostic and health management (PHM) systems are used to collect massive real-time data from the mechanical equipment, through the use of sensors that capture the asset behaviour over time. This technological advent creates a paradigm shift that opens up significant business opportunities. The direct costs and reputation loss caused by poor asset reliability can significantly impact organizational performance. The ability to predict an asset's reliability is therefore a core capability. 

Reliably estimating the RUL of an asset is an opportunity that decision makers need to seize in order to save costs. RUL predictions help *avoid unscheduled maintenance*, *increase equipment usage* as well as *improve operational safety*. The US Department of Energy found that predictive maintenance can **cut on-going maintenance costs by up to 30%, eliminate 75% of breakdowns, and reduce energy consumption by up to 20%**. In addition, a report by Deloitte University Press, <a href=" https://www2.deloitte.com/insights/us/en/focus/industry-4-0/manufacturing-ecosystems-exploring-world-connected-enterprises.html">Industry 4.0 and manufacturing ecosystems</a>, provides examples in which, for companies like Schneider Electric and Caterpillar, predictive maintenance and understanding root cause of failures **can offer millions of dollars in potential savings along with far fewer days of equipment downtime**.
RUL estimates provide decision makers with information that allows them to change operational characteristics (such as load) which in turn may prolong the life of the component. It also allows planners to account for upcoming maintenance and set in motion a logistics process that supports a smooth transition from faulty equipment to fully functional. Aircraft engines, medical equipment, power plants, etc. are some of the common examples of these types of equipment. 

In this challenge, we will be using aircraft engines Run-to-Failure simulated data, from the <a href="https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/">Prognostics Data Repository</a>.
The Dataset consists of a multivariate sensory data collected from the engines operating from normal to failure. The engine starts with different degrees of initial wear and manufacturing variation.  This wear and variation is considered normal, i.e., it is not considered a fault condition. 
There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. In additon, the data is contaminated with sensor noise.
The engine is operating normally at the start of the time serie, and develops a fault at some point during the serie. In the training set, the fault grows in magnitude until system failure. In the test set, the time series end some time prior to system failure. The RUL estimates are in units of time (cycles for engines)


# Presenting the data

The dataset is constructed of 26 dimensions/columns, wherein the first column is the unit number for different engines, the second column is the time index (cycle), the third through fifth columns are the settings of operational conditions, and the other 21 columns are simulated sensory data. 
The train data covers the Run-to-Failure process of 100 testing engines and consists of 20,631 data points. The test data contains the data collected in the running process with no failures, which is used to predict when the failure will occur—in other words, to estimate the RUL. The operational conditions of train and test are the same but have different initial states, which are caused by the variant initial wear and manufacture bias.





# Evaluation metric
The objective of the challenge is to predict the number of remaining operational cycles before failure in the test set, i.e., 
the number of operational cycles after the last cycle that the engine will continue to operate.

For results evaluation, estimated RULs are compared with actual RULs provided in the RUL column of the test data. In PHM context, it is generally desirable to have early RUL estimates rather than late RULs. Another criterion is the accuracy of prognostics model evaluated by **the Root Mean Squared Error** or **RMSE**, since it penalizes large errors severely, which will force the algorithm to forecast RUL as accurately as possible.

$$ RMSE = \sqrt{\frac{1}{n}\sum_{i=0}^n (y_i - \hat{y_i})^2} $$

# Loading the data

Data source <a href="https://c3.nasa.gov/dashlink/resources/139/">here.</a> 

In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor, GradientBoostingRegressor
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import os
import IPython.display as dis
%matplotlib inline

np.random.seed(8)
path = os.getcwd()

pd.options.display.max_columns = 100

In [54]:
train = pd.read_csv(os.path.join(path, "data", "train1.txt"), sep=' ')
test = pd.read_csv(os.path.join(path, "data", "test1.txt"), sep=' ')
train.head()

,unit,cycle,os1,os2,os3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1.0,1.0,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392.0,2388.0,100.0,39.06,23.4190,191.0
1,1.0,2.0,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392.0,2388.0,100.0,39.00,23.4236,190.0
2,1.0,3.0,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390.0,2388.0,100.0,38.95,23.3442,189.0
3,1.0,4.0,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392.0,2388.0,100.0,38.88,23.3739,188.0
4,1.0,5.0,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393.0,2388.0,100.0,38.90,23.4044,187.0


In [48]:
test.head()

,unit,cycle,os1,os2,os3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1.0,1.0,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,2388.04,9050.17,1.3,47.20,521.72,2388.03,8125.55,8.4052,0.03,392.0,2388.0,100.0,38.86,23.3735,142.0
1,1.0,2.0,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,2388.01,9054.42,1.3,47.50,522.16,2388.06,8139.62,8.3803,0.03,393.0,2388.0,100.0,39.02,23.3916,141.0
2,1.0,3.0,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,2388.05,9056.96,1.3,47.50,521.97,2388.03,8130.10,8.4441,0.03,393.0,2388.0,100.0,39.08,23.4166,140.0
3,1.0,4.0,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,554.07,2388.03,9045.29,1.3,47.28,521.38,2388.05,8132.90,8.3917,0.03,391.0,2388.0,100.0,39.00,23.3737,139.0
4,1.0,5.0,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,554.16,2388.01,9044.55,1.3,47.31,522.15,2388.03,8129.54,8.4031,0.03,390.0,2388.0,100.0,38.99,23.4130,138.0


# Exploratory Data Analysis

# Feature Egineering

# Modeling

In [66]:
y_train = train['RUL']
X_train = train.drop("RUL", axis=1)
y_test = test['RUL']
X_test = test.drop("RUL", axis=1)

In [67]:
scaler = preprocessing.StandardScaler()
X_train[X_train.drop('unit', axis=1).columns] = pd.DataFrame(scaler.fit_transform(X_train.drop('unit', axis=1)), columns=X_train.drop('unit', axis=1).columns)
X_test[X_train.drop('unit', axis=1).columns] = pd.DataFrame(scaler.fit_transform(X_test.drop('unit', axis=1)), columns=X_train.drop('unit', axis=1).columns)

In [68]:
X_train.shape, y_train.shape

((20631, 26), (20631,))

In [69]:
X_test.shape, y_test.shape

((13096, 26), (13096,))

In [70]:
X_train.head()

,unit,cycle,os1,os2,os3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1.0,-1.565170,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-1.0,0.141683,1.121141,-0.516338,-0.862813,0.0,-0.266467,0.334262,-1.058890,-0.269071,-0.603816,-1.0,-0.781710,0.0,0.0,1.348493,1.194427
1,1.0,-1.550652,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-1.0,0.141683,0.431930,-0.798093,-0.958818,0.0,-0.191583,1.174899,-0.363646,-0.642845,-0.275852,-1.0,-0.781710,0.0,0.0,1.016528,1.236922
2,1.0,-1.536134,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-1.0,0.141683,1.008155,-0.234584,-0.557139,0.0,-1.015303,1.364721,-0.919841,-0.551629,-0.649144,-1.0,-2.073094,0.0,0.0,0.739891,0.503423
3,1.0,-1.521616,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-1.0,0.141683,1.222827,0.188048,-0.713826,0.0,-1.539489,1.961302,-0.224597,-0.520176,-1.971665,-1.0,-0.781710,0.0,0.0,0.352598,0.777792
4,1.0,-1.507098,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-1.0,0.141683,0.714393,-0.516338,-0.457059,0.0,-0.977861,1.052871,-0.780793,-0.521748,-0.339845,-1.0,-0.136018,0.0,0.0,0.463253,1.059552


In [71]:
X_test.head()

,unit,cycle,os1,os2,os3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1.0,-1.429375,1.049295,1.005927,0.0,0.0,1.359276,-0.561495,-0.975674,-1.0,0.17566,0.209138,-0.539078,-0.720312,-1.0,-1.103591,-0.049542,-0.720590,-1.315031,-0.711659,-1.0,-0.463406,0.0,0.0,-0.229408,0.448866
1,1.0,-1.410527,-1.220748,-1.034755,0.0,0.0,-1.908502,0.070116,-1.392836,-1.0,0.17566,1.603612,-1.061366,-0.348673,-1.0,0.427727,0.736727,-0.193647,0.065977,-1.570037,-1.0,0.347276,0.0,0.0,0.899934,0.664042
2,1.0,-1.391679,0.141278,0.325700,0.0,0.0,-0.037636,-0.231698,-0.507676,-1.0,0.17566,0.517390,-0.364982,-0.126564,-1.0,0.427727,0.397202,-0.720590,-0.868436,0.629340,-1.0,0.347276,0.0,0.0,1.323438,0.961247
3,1.0,-1.372830,1.911912,-0.014414,0.0,0.0,-0.087526,-0.795350,0.251888,-1.0,0.17566,0.458675,-0.713174,-1.147041,-1.0,-0.695239,-0.657113,-0.369295,-0.593609,-1.177045,-1.0,-1.274088,0.0,0.0,0.758766,0.451244
4,1.0,-1.353982,0.640688,-0.014414,0.0,0.0,0.087088,-0.181729,-0.420954,-1.0,0.17566,0.590783,-1.061366,-1.211750,-1.0,-0.542108,0.718857,-0.720590,-0.923402,-0.784053,-1.0,-2.084769,0.0,0.0,0.688182,0.918450


In [72]:
rg = GradientBoostingRegressor()
rf = RandomForestRegressor()

In [79]:
rf.fit(X_train, y_train)
y_pred_test = rf.predict(X_test)
y_pred_train = rf.predict(X_train)

In [81]:
rmse = sqrt(mean_squared_error(y_train, y_pred_train))
print(rmse)

7.072940843240117


In [84]:
relative_mae = mean_absolute_error(y_train, y_pred_train) / np.mean(y_train)
print(relative_mae)

0.0399876089388


In [80]:
r2 = r2_score(y_train, y_pred_train)
print(r2)

0.989455588661


In [82]:
rmse = sqrt(mean_squared_error(y_test, y_pred_test))
print(rmse)

61.816856884282984


In [85]:
relative_mae = mean_absolute_error(y_test, y_pred_test) / np.mean(y_test)
print(relative_mae)

0.342152256173


In [83]:
r2 = r2_score(y_test, y_pred_test)
print(r2)

-0.0985903649829
